# Search Evaluation 2 - Vector Retrieval

Reference [LLM Zoomcamp 3.3.4 - Evaluating Vector Retrieval
](https://www.youtube.com/watch?v=VRprIm9-VV8)

# Step 0: Start up docker for ElasticSearch
```
docker run -it \
    --rm \
    --name elasticsearch \
    -m 2GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

# Step 1: Load Documents into ElasticSearch

In [1]:
import json
import numpy as np
np.float_ = np.float64
from tqdm.auto import tqdm
import pandas as pd
from elasticsearch import Elasticsearch

import numpy as np
np.float_ = np.float64

from sentence_transformers import SentenceTransformer

# download documents
!wget -nc https://raw.githubusercontent.com/slavaheroes/llm-zoomcamp/refs/heads/homeworks/03-vector-search/eval/documents-with-ids.json

# load documents.json and flatten them
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)


es_client = Elasticsearch('http://localhost:9200')

# create mapping
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector", 
                "dims": 384, 
                "index": True, 
                "similarity": "cosine"
                },
            "text_vector": {
                "type": "dense_vector", 
                "dims": 384, 
                "index": True, 
                "similarity": "cosine"
                },
            "question_text_vector": {
                "type": "dense_vector", 
                "dims": 384, 
                "index": True, 
                "similarity": "cosine"
                },
        }
    }
}

# create index
index_name = "course-questions"

es_client.indices.delete(index=index_name,ignore_unavailable=True)
es_client.indices.create(index=index_name,body=index_settings)


# initialize embeddings model
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

# create embeddings
for doc in tqdm(documents):
    try:
        question = doc['question'] 
        text = doc['text'] 
        qt = question +' ' + text

        doc['question_vector'] = model.encode(question)
        doc['text_vector'] = model.encode(text)
        doc['question_text_vector'] = model.encode(qt)
    except Exception as e:
        print(e)

# add documents to index
for doc in tqdm(documents):
    try: 
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

File ‘documents-with-ids.json’ already there; not retrieving.



  0%|          | 0/948 [00:00<?, ?it/s]

  0%|          | 0/948 [00:00<?, ?it/s]

# Step 2: Evaluate 
#### Step 2.1: Define functions

In [2]:
# define evaluation functions

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

# define search function to evaluate
def elastic_search_knn(field, vector, course):
    knn = {
            "field": field,
            "query_vector": vector,
            "k": 5,
            "num_candidates": 1000,
            "filter": {
                "term": {
                    "course": course
                }
            }
        }


    query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name, 
        body=query, 
        )
    
    result_docs = []
    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit['_source'])

    return result_docs

def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)
    
    return elastic_search_knn('question_vector', v_q, course)

def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)
    
    return elastic_search_knn('text_vector', v_q, course)

def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)
    
    return elastic_search_knn('question_text_vector', v_q, course)

In [ ]:
# example on how to use search function
search_term = 'I just discovered the course. Can I still join?'
search_term_vector = model.encode(search_term)
course = 'data-engineering-zoomcamp'

elastic_search_knn('question_vector', search_term_vector, course)

#### Step 2.2: Evaluate

In [4]:
# load ground truth dataset
fn = 'https://raw.githubusercontent.com/slavaheroes/llm-zoomcamp/refs/heads/homeworks/03-vector-search/eval/ground-truth-data.csv'
ground_truth = pd.read_csv(fn).to_dict(orient='records')

In [5]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.773071104387292, 'mrr': 0.6666810748505158}

In [6]:
evaluate(ground_truth, text_vector_knn)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.8286146531229739, 'mrr': 0.7062315395144454}

In [7]:
evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.9172249837907932, 'mrr': 0.824306606152295}